# Compact version of explicit scheme - with explicit solver (does not work)
Just the same as before, only now with less output
* adding "getC" to the definitons
* include "S" as variable
* checking out the implicit solver (seems to work)

## Definitions


In [1]:
import openmesh as om
import numpy as np
import matplotlib.pyplot as plt

mesh = om.read_trimesh('T4.off')
V=mesh.points()
X=V[:,0]
Y=V[:,1]

def initActiveNodes():
    ActiveNodeList = []
    for vh in mesh.vertices():
        k = vh.idx()
        if V[k,0] == 0 or V[k,1] == 1:
            ActiveNodeList.append(vh)
    return ActiveNodeList

def initActiveFaceList():
    ActiveFaceList = []

    i=0
    for fh in mesh.faces():
        counter = 0
    
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                # print(vh.idx(),' in ', fh.idx())                
                counter = counter + 1
            
        if counter == 2:
            ActiveFaceList.append(fh)

    return ActiveFaceList

def init_TS_nodes():
    T = np.zeros(len(V))+7
    S = np.zeros(len(V))+7

    for vh in ActiveNodeList:    
        k = vh.idx()
        if V[k,0] == 0:
            T[k] = 0
            S[k] = 0
        elif V[k,1] == 1:
            T[k] = V[k,0]
            S[k] = 0
    
    return T, S

def init_TS_faces():
    Tf = np.zeros(len(mesh.faces()))+7
    Sf = np.zeros(len(mesh.faces()))+7

    for fh in ActiveFaceList:
        T_ABC = 0
        S_ABC = 0
        counter = 0
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                T_ABC = T_ABC + T[vh.idx()]
                S_ABC = S_ABC + S[vh.idx()]            
                counter = counter + 1
            
        if counter != 2:
            print('ERR at face', fh.idx())
        else:
            Tf[fh.idx()] = T_ABC / 2
            Sf[fh.idx()] = S_ABC / 2
            
    return Tf, Sf
    
# routine to get active face indices    
def getActiveFaceIndices():
    I_list = []
    for fh in ActiveFaceList:
        I_list.append(fh.idx())
        
    return I_list

    
def getActiveNodeIndices():
    I_list = []
    for fh in ActiveNodeList:
        I_list.append(fh.idx())

    return I_list


def getProcessedFaceIndices():
    I_list = []
    for fh in ProcessedFaceList:
        I_list.append(fh.idx())
        
    return I_list

def getObsoleteFaceIndices():
    I_list = []
    for fh in ObsoleteFaceList:
        I_list.append(fh.idx())
        
    return I_list

def get_T_C_dummy(T_A, T_B):
    return (T_A + T_B)/2 + 10

# ... update faces: don inline    

def getAngle(v1, v2):
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))    
    return angle

def getGrad2(I, U):
    m = np.zeros((2, 2))
    b = np.zeros((2, 1))

    for k in range(2):
        m[k,0] = V[I[k+1],0]-V[I[0],0]
        m[k,1] = V[I[k+1],1]-V[I[0],1]
        b[k] = U[k+1]-U[0]
                
    g = np.linalg.solve(m, b)    
    c = np.linalg.cond(m)

    return c, g

def test():
    print(S)
    print(T)    

def get_S_C(I): # savety version    
    Ta=T[I[0]]
    Tb=T[I[1]]
    Tc=T[I[2]]
    
    
    Sa=S[I[0]]
    Sb=S[I[1]]
    Sc=S[I[2]]
    # Sc=S[I[2]]
    Sc=max(S[I[0]],S[I[1]])+0.01    
    
    U=np.zeros(3)
    U[0]=T[I[0]]
    U[1]=T[I[1]]
    U[2]=T[I[2]]
     
    g=getGrad2(I, U)
    gnorm = np.linalg.norm(g)
    gs = g / (gnorm+1e-9)

    Yc=V[I[2],1]

##########################################################################

    
    m=np.argmax(S[I[0:2]])
    Vm=V[I[m],0:2]
    Vc=V[I[2],0:2]
    
    cm = S[m]-np.dot(gs.transpose(), Vm)
    Sm = np.dot(gs.transpose(), Vc) + cm

    # return Sm
    return Sc


def getC(I):
    A = V[I[0],0:2]
    B = V[I[1],0:2]
    C = V[I[2],0:2]    
    TA = T[I[0]]
    TB = T[I[1]]
    SA = T[I[0]]
    SB = T[I[1]]
    
    a = sum((B-C)**2)**(1/2)
    b = sum((A-C)**2)**(1/2)
    c = sum((A-B)**2)**(1/2)
    
    # firstround = 1
    TC=min(TA,TB)
    TC=(TA+TB)/2+0.01

    p1 = abs(TC-TA)/b
    p2 = abs(TC-TB)/a
    p3 = abs(TA-TB)/c
    
    alpha = getAngle(B-A, C-A)
    beta = getAngle(A-B, C-B)
    gamma = getAngle(A-C, B-C)

    asum=alpha+beta+gamma
    if asum != np.pi:
        print('ERR: angles do not sum up to pi: %.2f' % asum)
      

    YC = V[I[2],1] # C[1]
    # SC = get_S_C(I)        
    # SC = (SA+SB+1e-5)/2    
    SC = max(SA,SB)+0.01
    
    fC = SC / (YC+1e-5)
    
    tfrac=abs(TA-TB)/(c*fC+1e-5)
    if tfrac > 0.99:
        print('ERR: tfrac greater 1: %.2f' %tfrac)
        theta = np.pi
    else:
        theta = np.arcsin(tfrac)
    
    counter = 0
    if p2 < fC:
        counter = counter + 1
    else:
        print('ERR: Huygens principle')

    if theta < beta:
        counter = counter + 1
    else:
        print('ERR: Evaluation order')
        
    if theta + alpha < np.pi / 2:
        counter = counter + 1
    else:    
        print('ERR: Causality: theta: %.2f' %theta, ', alpha: %.2f' % alpha)        

    if counter == 3:
        HC = 1/np.sin(gamma)*np.sqrt(p1**2 - 2*p1*p2*np.cos(gamma)+p2**2)
    else:
        HC = min(TC, TA+b*fC, TB +a*fC)
        
    TC = HC
    SC = get_S_C(I)

    return TC, SC


ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()
    
T, S = init_TS_nodes()
Tf, Sf = init_TS_faces()

I_active_faces = getActiveFaceIndices()
print('active faces:', I_active_faces)

I = [0, 5, 30]
TC, SC = getC(I)

active faces: [1, 4, 5, 8, 11, 22, 40, 74, 75]


## Running

In [2]:
ProcessedFaceList = []
ObsoleteFaceList = []

ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()
global_I = I

# print('len: list(I[K]) \t \t (id_A, id_B) -> id_C \n \n \n')
j=0
while len(ActiveFaceList)>0:
    j=j+1
    
    # (0) update the index
    I_active_faces = getActiveFaceIndices()

    I = np.array(I_active_faces)
    K = np.argsort(Tf[I], axis=0)

    # print('(len)', len(I_active_faces),': (f)', list(I[K]))
    
    # (1) obtain id of next face
    next_face_id = (I[K])[0]
    fh = mesh.face_handle(next_face_id)
    
    # quick check whether the face is obsolete,
    # which is a normal situation that should not be handled as an error
    # but by moving on to the next face

    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            counter = counter + 1    
    if counter == 3:
        # print('remove obsolete face:', fh.idx())
        ObsoleteFaceList.append(fh)
        ActiveFaceList.remove(fh)
        continue
            
    # (2) calculate value of outgoing vertex
    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            # print(vh.idx(),'ingoing')
            if counter == 0:
                counter = counter + 1
                id_A = vh.idx()
                T_A = T[vh.idx()] # not needed ?
                S_A = S[vh.idx()]                
            elif counter == 1:
                counter = counter + 1
                id_B = vh.idx()
                T_B = T[vh.idx()]
                S_B = S[vh.idx()]
            else:
                print('ERR: too many ingoing nodes')
                
        else:
            # print(vh.idx(),'outgoing')        
            id_C = vh.idx()
            vh_C = vh

######################################################################################################

    if counter == 2:
        I_ABC = [id_A, id_B, id_C]
        # I = np.array(I)
        global_I = I_ABC

        # T_C_dummy = get_T_C_dummy(T_A, T_B)        
        T_C, S_C = getC(I_ABC)
        
        # print('T', I_ABC,' : %.2f' %T_C)        
        # T_C = T_C_dummy        
        # T_C = T_C + j / 1000
        
        T[id_C] = T_C
        S[id_C] = S_C        
    else:
        print('ERR: missing inputs for calculating T_C')
        # T[id_C] = j
        
    # T[id_C] = j
    
    # (3) handle the list of active faces
    if (not vh_C in ActiveNodeList):
        ActiveNodeList.append(vh_C) # also ! 
    else:
        print('node already listed:', vh_C.idx())
    
    ActiveFaceList.remove(fh)
    ProcessedFaceList.append(fh)

    # (3b) agregate new faces to the list
    f_list = []
    print('(', id_A, ',', id_B, ') ->', id_C,'\t \t',len(I_active_faces),': ', list(I[K]))    
    for fvh in mesh.vf(vh_C):
        f_list.append(fvh.idx())
        # print('fv', fvh.idx())
        if (not fvh in ActiveFaceList) and (not fvh in ProcessedFaceList):
            counter = 0
            for vh in mesh.fv(fvh):
                if vh in ActiveNodeList:
                    counter = counter + 1
        
            if counter == 2:
                ActiveFaceList.append(fvh)
                # print('new face:', fvh.idx())
                
                counter = 0
                for vh in mesh.fv(fvh):
                    if vh in ActiveNodeList:
                        if counter == 0:
                            T_A = T[vh.idx()]
                            counter = counter + 1
                        elif counter == 1:
                            T_B = T[vh.idx()]
                    Tf[fvh.idx()] = (T_A + T_B)/2
            
            elif counter == 3:
                print('face duplicate registered:', fvh.idx())                
        
print('\n T: \n', T)
print('\n Tf: \n', Tf)

( 5 , 0 ) -> 30 	 	 9 :  [5, 8, 11, 74, 75, 1, 40, 22, 4]
( 5 , 10 ) -> 34 	 	 10 :  [8, 11, 74, 75, 0, 9, 1, 40, 22, 4]
( 15 , 10 ) -> 38 	 	 11 :  [11, 74, 75, 7, 0, 9, 33, 1, 40, 22, 4]
ERR: Huygens principle
( 20 , 25 ) -> 46 	 	 12 :  [74, 75, 7, 43, 0, 9, 6, 33, 1, 40, 22, 4]
ERR: Huygens principle
( 20 , 15 ) -> 42 	 	 13 :  [75, 76, 7, 43, 0, 9, 6, 33, 1, 35, 40, 22, 4]
ERR: Causality: theta: 0.38 , alpha: 1.37
( 42 , 46 ) -> 21 	 	 13 :  [44, 7, 43, 0, 9, 73, 6, 33, 1, 35, 40, 22, 4]
ERR: Causality: theta: 0.00 , alpha: 2.75
( 30 , 0 ) -> 1 	 	 12 :  [0, 9, 72, 73, 6, 33, 1, 35, 42, 40, 22, 4]
( 42 , 21 ) -> 16 	 	 11 :  [72, 73, 6, 33, 32, 1, 35, 42, 40, 22, 4]
( 38 , 34 ) -> 11 	 	 11 :  [6, 33, 32, 68, 71, 1, 35, 42, 40, 22, 4]
( 34 , 11 ) -> 6 	 	 12 :  [34, 33, 32, 68, 70, 71, 1, 35, 42, 40, 22, 4]
ERR: tfrac greater 1: 1.48
ERR: Evaluation order
ERR: Causality: theta: 3.14 , alpha: 1.37
( 21 , 16 ) -> 43 	 	 11 :  [68, 70, 71, 30, 31, 1, 35, 42, 40, 22, 4]
( 21 , 43 ) ->

## Checksum

In [3]:
I_active_nodes = getActiveNodeIndices()
print('active nodes:', I_active_nodes)


active nodes: [0, 5, 10, 15, 20, 25, 26, 27, 28, 29, 30, 34, 38, 46, 42, 21, 1, 16, 11, 6, 43, 47, 39, 35, 31, 2, 17, 22, 44, 40, 12, 48, 18, 13, 23, 45, 41, 49, 36, 19, 24, 14, 37, 7, 32, 3, 8, 33, 9, 4]


In [4]:

I_active_nodes = getActiveNodeIndices()
print('active nodes (usorted):', I_active_nodes)
print(len(I_active_nodes),': ',np.sort(I_active_nodes))

I_faces = getProcessedFaceIndices()
I_faces_obsolete = getObsoleteFaceIndices()
print('check completeness:', len(I_faces), len(I_faces_obsolete))
print('faces, processed:', np.sort(I_faces))
print('faces, obsolete:', np.sort(I_faces_obsolete))


active nodes (usorted): [0, 5, 10, 15, 20, 25, 26, 27, 28, 29, 30, 34, 38, 46, 42, 21, 1, 16, 11, 6, 43, 47, 39, 35, 31, 2, 17, 22, 44, 40, 12, 48, 18, 13, 23, 45, 41, 49, 36, 19, 24, 14, 37, 7, 32, 3, 8, 33, 9, 4]
50 :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
check completeness: 40 40
faces, processed: [ 0  2  3  5  6  8 10 11 12 13 15 17 18 21 24 31 34 36 38 41 44 45 49 50
 51 53 58 59 61 62 63 64 65 66 68 70 72 74 75 79]
faces, obsolete: [ 1  4  7  9 14 16 19 20 22 23 25 26 27 28 29 30 32 33 35 37 39 40 42 43
 46 47 48 52 54 55 56 57 60 67 69 71 73 76 77 78]


In [7]:
I = [25, 20, 46]
print('\n T[I] \n',T[I], '\n S[I]: \n', S[I], '\n V[I]: \n', V[I,0:2])

def residual(u):
    I = I_global
    # u = np.zeros(2) # strange duplication needed
    # uu = uuu
    T[I[2]]=u[0]
    S[I[2]]=u[1]
    
    m = np.zeros((2, 2))
    b = np.zeros((2, 1))

    for k in range(2):
        m[k,0] = V[I[k],0]-V[I[2],0]
        m[k,1] = V[I[k],1]-V[I[2],1]
        b[k] = T[I[k]]-T[I[2]]
    g = np.linalg.solve(m, b)

    gnorm = np.linalg.norm(g)
    Ybar=sum(V[I,1])/3
    Sbar=sum(S[I])/3
    rT = gnorm - Ybar / (Sbar + 1e-7)
        
    # print('g:', g, '\n Ybar:', Ybar, '\n Sbar:', Sbar, '\n rT:', rT, '\n\n\n')    
    # proceed with the calculation of rS
    m=np.argmin(S[I[0:2]])
    Vm=V[I[m],0:2]
    Vc=V[I[2],0:2]
    
    # print('Vm', Vm, 'Vc', Vc, '\n\n\n')    
    gs = g / (gnorm+1e-9)
    cm = S[I[m]] - np.dot(gs.transpose(), Vm)
    Sm = np.dot(gs.transpose(), Vc) + cm    
    rS = S[I[2]] - Sm
    
    r = [rT, rS[0]]
    
    return r

def cost(U):
    R=residual(U)
    c=np.inner(R,R)
    # print(c)
    return c


 T[I] 
 [0. 0. 1.] 
 S[I]: 
 [0.  0.  0.4] 
 V[I]: 
 [[0.         1.        ]
 [0.         0.80000001]
 [0.5        0.89999998]]


In [10]:
I_global = [25, 20, 46]
I=I_global
U = np.zeros(2)

U[0]=T[I[2]]
U[1]=S[I[2]]

U[0]=1.0
U[1]=0.4
T[I[2]]=U[0]
S[I[2]]=U[1]

print(U)

r = residual(U)
c = cost(U)
print(U, c, r)


[1.  0.4]
[1.  0.4] 22.572451623140154 [-4.749994907701497, -0.09999999974999996]


In [11]:
import scipy.optimize as opt
import time

t0 = time.time()
res = opt.minimize(cost, U, method='powell', tol=1e-10, options={'maxiter': 2000, 'disp': True})
t1 = time.time()

print('\n t1-t0: = %.1f' % float(t1 - t0),'\n res:', res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 423

 t1-t0: = 0.1 
 res:    direc: array([[-8.35345961e-02,  1.60375777e-02],
       [-2.77055051e-08,  4.13108167e-09]])
     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 423
     nit: 9
  status: 0
 success: True
       x: array([2.69999837, 0.5       ])
